In [1]:
# Jan 30, 2020
# Joined the 5 classification tables available
# to be evaluated:
    # join is left by default
    # because I adopt cit_received as the leftmost data
        # this dataset keep data on the patents who received citation
    # I can make the first join as an 'outer' operation between cit_received and cit_made
    # To be complete, I can begin with patent.csv.gz and make the first two joins with 'outer' type
#     Ill git add now and try this
    

# Jan 29, 2020
# many errors appeared after joining the dataset 
# i rewritten code into simpler scripts
# this script joins variables together to be processed later

# they are:
#     - cit_delay - reads clean_patent and clean_uspatentcitation and calculates delay by patent
#     - cit_made - reads clean_uspatentcitation and calculates citations made 
#     - cit_received - reads clean_uspatentcitation and calculates citations received
#     - cit_tree - reads clean_uspatentcitation and cit_made and calculates parent_citation
#     - generalit - reads wipo and clean_uspatentcitation and calculates generality
#     - originality - reads wipo and clean_uspatentcitation and calculates originality
#     - wipo_first_class - reads wipo and generates wipo_first_class

# it is now way faster, but I cannot know for sure if this is a consequence of coding or something changed in 
# infrastructure. 

# since class is a variable that also appear in non-cited patents, I'll include them in the final step


# Jan, 20th 2020
# join delay and tree data to form the analysis dataset


# In[2]:

In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.delayed import delayed


def data_read(file, names, usecols, dtype):
    df = dd.read_parquet(file, compression='gzip', names=names, usecols=usecols, dtype=dtype, index_col='id', header=0)
    dfs.append(df)

In [3]:
dst='data/dataset.parquet.gz'

patent='data/cleanpatent.parquet.gz'
received='data/cit_received.parquet.gz'
made='data/cit_made.parquet.gz'
received_delay='data/cit_received_delay.parquet.gz'
made_delay='data/cit_made_delay.parquet.gz'
# cit_tree = 'data/cit_tree.csv.gz'
originality = 'data/originality.parquet.gz'
generality = 'data/generality.parquet.gz'
wipo = 'data/wipo.parquet.gz'
centrality='data/centrality_pagerank.csv'
int_ext_cit='data/int_ext_cit.parquet.gz'

In [4]:
dfs=[]

#adopting patent.csv as the reference of patents
df = dd.read_parquet(patent, compression='gzip', dtype=object)
df['num_claims']=df['num_claims'].astype(float) #int does not handle NAN values, so using float instead
# df=df.set_index('id')

In [5]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 2 entries, date to num_claims
dtypes: float64(1), int64(1)

In [6]:
#outer join to citation received
#there are patents who received citations that are not in patent.csv.gz
#for example, the very ancient ones

In [7]:
df2 = dd.read_parquet(received, compression='gzip')
df=df.join(df2, how='outer')
df['cit_received']=df['cit_received'].fillna(0)

In [8]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 3 entries, date to cit_received
dtypes: float64(1), int64(2)

In [9]:
#i dont see any reason for patents making citations not to appear in patent.csv.gz
#so from this data on, the list of patents is stable
#the number of rows should not change (check)
df2 = dd.read_parquet(made, compression='gzip')
df=df.join(df2, how='outer')

In [10]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 4 entries, date to cit_made
dtypes: float64(1), int64(3)

In [11]:
df2 = dd.read_parquet(originality, compression='gzip')
df=df.join(df2, how='outer')

In [12]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 5 entries, date to originality
dtypes: float64(2), int64(3)

In [13]:
df2 = dd.read_parquet(generality, compression='gzip')
df=df.join(df2, how='outer')

In [14]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 6 entries, date to generality
dtypes: float64(3), int64(3)

In [15]:
df2 = dd.read_parquet(wipo, compression='gzip')
df=df.join(df2, how='outer')

In [16]:
df2 = dd.read_parquet(int_ext_cit, compression='gzip', dtype=object)
# df.set_index('index', inplace=True)
df=df.join(df2, how='outer')

In [17]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 10 entries, date to far_external
dtypes: category(2), float64(3), int64(5)

In [18]:
df2 = dd.read_csv(centrality, dtype=object)
df2=df2.set_index('id')
df=df.join(df2, how='outer')

In [19]:
# adding some new files
inventor='data/inventor.parquet.gz'
application='data/application.parquet.gz'
foreign_cit='data/foreigncitation.parquet.gz'
foreign_priority='data/foreign_priority.parquet.gz'
pat_govt='data/df_pat_govt.parquet.gz'
pat_assignee='data/pat_assignee.parquet.gz'
otherref_cit = 'data/otherref_cit.parquet.gz'

In [20]:
df2 = dd.read_parquet(inventor, dtype=object)
df2=df2.set_index('id')
df=df.join(df2, how='outer')

In [ ]:
df2 = dd.read_parquet(application)
df2=df2.set_index('id')
df=df.join(df2, how='outer')

In [ ]:
df2 = dd.read_parquet(foreign_cit)
df2=df2.set_index('patent_id')
df=df.join(df2, how='outer')

In [ ]:
df2 = dd.read_parquet(foreign_priority, dtype=object)
df2=df2.set_index('patent_id')
df=df.join(df2, how='outer')

In [ ]:
df2 = dd.read_parquet(pat_govt, dtype=object)
df=df.join(df2, how='outer')

In [ ]:
df2 = dd.read_parquet(pat_assignee, dtype=object)
#df2=df2.set_index('patent_id')
df=df.join(df2, how='outer')

In [ ]:
df2 = dd.read_parquet(otherref_cit, dtype=object)
#df2=df2.set_index('patent_id')
df=df.join(df2, how='outer')

In [ ]:
df=df.compute()

In [ ]:
df.info()

In [ ]:
df.sample(n=10)

In [ ]:
df.describe()

In [ ]:
df.describe(include=[np.object]).transpose()

In [ ]:
df.to_parquet(dst, compression='gzip')

In [ ]:
# In[32]:


# this code includes WIPO in the dataset
# as class is also part of non-cited patents, I'm not including in here.
# in any case, for some reason this merge could not be done by the method used in the previous
# there is a bug when using read_csv.gz using dtype and indexing. 
# it is a known bug in the community but i do not know why the previous join did not accused the issue
# the method below corrects the issue 

# names=['index', 'id', 'field_id']
# dtype={'id':object}
# usecols=['id', 'field_id']
# # data_read(wipo, names, usecols, dtype)
# wipo = pd.read_csv.gz(wipo, names=names, usecols=usecols, dtype=dtype, header=0)
# wipo['id']=wipo['id'].astype(str)
# wipo=wipo.set_index('id')
# wipo.info()

# dfs.append(wipo)